### *IPCC SR15 scenario assessment*

<img style="float: right; height: 80px; padding-left: 20px;" src="../_static/IIASA_logo.png">
<img style="float: right; height: 80px;" src="../_static/IAMC_logo.jpg">

# Annual global emissions characteristics <br /> of long-lived climate forcers

This notebook plots the development of carbon dioxide emissions by different subsectors
for **Figure 2.6** in the IPCC's _"Special Report on Global Warming of 1.5°C"_.

The scenario data used in this analysis can be accessed and downloaded at [https://data.ene.iiasa.ac.at/iamc-1.5c-explorer](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer).

## Load `pyam` package and other dependencies

In [ ]:
import pandas as pd
import numpy as np
import warnings
import io
import itertools
import yaml
import math
import matplotlib.pyplot as plt
plt.style.use('style_sr15.mplstyle')
%matplotlib inline
import pyam

from utils import boxplot_by_cat

## Import scenario data, categorization and specifications files

The metadata file with scenario categorisation and quantitative indicators can be downloaded at [https://data.ene.iiasa.ac.at/iamc-1.5c-explorer](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer).  
Alternatively, it can be re-created using the notebook `sr15_2.0_categories_indicators`.

The last cell of this section loads and assigns a number of auxiliary lists as defined in the categorization notebook.

In [ ]:
sr1p5 = pyam.IamDataFrame(data='../data/iamc15_scenario_data_world_r2.0.xlsx')

In [ ]:
sr1p5.load_meta('sr15_metadata_indicators.xlsx')

In [ ]:
with open("sr15_specs.yaml", 'r') as stream:
    specs = yaml.load(stream, Loader=yaml.FullLoader)

rc = pyam.run_control()
for item in specs.pop('run_control').items():
    rc.update({item[0]: item[1]})
cats = specs.pop('cats')
all_cats = specs.pop('all_cats')
subcats = specs.pop('subcats')
all_subcats = specs.pop('all_subcats')
plotting_args = specs.pop('plotting_args')
marker= specs.pop('marker')

## Downselect scenario ensemble to categories of interest for this assessment

In [ ]:
cats.remove('Above 2C')

In [ ]:
years = [2020, 2030, 2050, 2100]

In [ ]:
sr1p5.meta.rename(columns={'Kyoto-GHG|2010 (SAR)': 'kyoto_ghg_2010'}, inplace=True)

In [ ]:
df = sr1p5.filter(kyoto_ghg_2010='in range', category=cats, year=years)

## Set specifications for filter and plotting

In [ ]:
save_name = 'output/fig2.6{}.{}'

In [ ]:
filter_args = dict(df=sr1p5, category=cats, marker=None, join_meta=True)

In [ ]:
def plotting_args(name, filetype='png', hlines=[0]):
    return {'categories': cats, 'column': 'category', 'years': years, 'add_marker': marker,
            'hlines': hlines, 'save': save_name.format(name, filetype)}

In [ ]:
data = []

## Plot different emissions pathways by category

In [ ]:
ghg = (
    df.filter(variable='Emissions|Kyoto Gases (AR4-GWP100)')
    .rename(unit={'Mt CO2-equiv/yr': 'Mt CO2e/yr'})
    .convert_unit('Mt CO2e/yr','Gt CO2e/yr')
    .timeseries()
)

In [ ]:
name = 'kyoto'
_data = pyam.filter_by_meta(ghg, **filter_args)
fig = boxplot_by_cat(_data, ylabel='Global Kyoto-GHG emissions (GtCO2e AR4GWP)',
                     **plotting_args('a_{}'.format(name)))
_data['species'] = name
data.append(_data)

In [ ]:
co2 = (
    df.filter(variable='Emissions|CO2')
    .convert_unit('Mt CO2/yr', 'Gt CO2/yr')
    .timeseries()
)

In [ ]:
name = 'co2_net_total'
_data = pyam.filter_by_meta(co2, **filter_args)
fig = boxplot_by_cat(_data, ylabel='Global CO2 emissions (GtCO2)',
                     **plotting_args('b_{}'.format(name)), legend=False)
_data['species'] = name
data.append(_data)

In [ ]:
co2_afolu = (
    df.filter(variable='Emissions|CO2|AFOLU')
    .convert_unit('Mt CO2/yr', 'Gt CO2/yr')
    .timeseries()
)

In [ ]:
name = 'co2_afolu'
_data = pyam.filter_by_meta(co2_afolu, **filter_args)
fig = boxplot_by_cat(_data, ylabel='Global CO2 emissions from AFOLU (GtCO2)',
                     **plotting_args('c_{}'.format(name)), legend=False)
_data['species'] = name
data.append(_data)

In [ ]:
n2o = (
    df.filter(variable='Emissions|N2O')
    .convert_unit('kt N2O/yr', 'Mt N2O/yr', factor=1/1000)
    .timeseries()
)

In [ ]:
name = 'n2o'
_data = pyam.filter_by_meta(n2o, **filter_args)
fig = boxplot_by_cat(_data, ylabel='Global N2O emissions (MtN2O)',
                     **plotting_args('d_{}'.format(name), hlines=None),
                     legend=False)
_data['species'] = name
data.append(_data)

In [ ]:
co2_ene = (
    df.filter(variable='Emissions|CO2|Energy and Industrial Processes')
    .convert_unit('Mt CO2/yr', 'Gt CO2/yr')
    .timeseries()
)

In [ ]:
name = 'co2_ffi'
_data = pyam.filter_by_meta(co2_ene, **filter_args)
fig = boxplot_by_cat(_data, ylabel='Global CO2 emissions from fossil fuels and industry (GtCO2)',
                     **plotting_args('e_{}'.format(name)), legend=False)
_data['species'] = name
data.append(_data)

In [ ]:
co2_supply = (
    df.filter(variable='Emissions|CO2|Energy|Supply')
    .convert_unit('Mt CO2/yr', 'Gt CO2/yr')
    .timeseries()
)

In [ ]:
name = 'co2_supply'
_data = pyam.filter_by_meta(co2_supply, **filter_args)
fig = boxplot_by_cat(_data, ylabel='Global CO2 emissions from energy supply (GtCO2)',
                     **plotting_args('f_{}'.format(name)), legend=False)
_data['species'] = name
data.append(_data)

In [ ]:
co2_demand = (
    df.filter(variable='Emissions|CO2|Energy|Demand')
    .convert_unit('Mt CO2/yr', 'Gt CO2/yr')
    .timeseries()
)

Exclude scenarios that report the CO2 emissions from energy demand as zero from this plot.

In [ ]:
co2_demand[co2_demand.apply(lambda x: max(x), axis=1) < 0.1]

In [ ]:
co2_demand = co2_demand[co2_demand.apply(lambda x: max(x), axis=1) > 0.1]

In [ ]:
name = 'co2_demand'
_data = pyam.filter_by_meta(co2_demand, **filter_args)
fig = boxplot_by_cat(_data, ylabel='Global CO2 emissions from energy demand (GtCO2)',
                     **plotting_args('g_{}'.format(name)), legend=False)
_data['species'] = name
data.append(_data)

## Export timeseries data to `xlsx`

In [ ]:
data = pd.concat(data).set_index(['species', 'category', 'marker'], append=True)

In [ ]:
data.head()

In [ ]:
data.reset_index().to_excel('output/fig2.6_data_table.xlsx')